# 第一次CV实验作业

这次实验中，我们要做的是实现对于传送带上红色物块和黄色物块的自动计数。那么我们如何设置呢？

首先，从人工识别的角度来看，我们通常会选择一个特定区域，观察是否有物块出现，并在物块离开画面时进行一次计数。基于这一过程，我们可以将其分解为以下几个关键步骤：
* 设定ROI，即我们想让计算机观测计数的区域。
* 分别提取红色和黄色物块的颜色范围，确保能够将物块与背景或其他物体区分开来。
* 在视频中设置ROI，利用提取的颜色范围检测场景中是否有物块出现，并在物块离开画面时进行计数。

*为了下序步骤有效地调参，我们需要先对视频进行截图操作*

## 视频截图

在这一步骤，直接调用`VideoCapture`函数打开视频即可。我们通过设置快捷键s来进行截图，设置快捷键q来进行退出操作。

In [1]:
import cv2
import time

def save_video_image(video_path:str):
    cap = cv2.VideoCapture(video_path)
    picture_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End")
            break
        cv2.imshow("Video", frame)
        time.sleep(0.02)
        pressed_key = cv2.waitKey(1)&0xFF
        if pressed_key == ord('q'):
            break
        elif pressed_key == ord('s'):
            picture_path = f"picture_{picture_index}.jpg"
            picture_index += 1
            cv2.imwrite(picture_path, frame)
        else:
            continue
    cap.release()
    cv2.destroyAllWindows()

In [2]:
save_video_image('data/Belt1.mp4') # 该平台运行，服务会dead

## 设定ROI

为了设置所需的ROI，我们可以利用OpenCV中的`createTrackbar`函数创建滑动条，并通过`getTrackbarPos`函数获取滑动条的当前值。通过将这些参数应用到图像上，可以实时查看不同参数设置下的图像效果。

In [3]:
import cv2

def callback(a):
    pass

def get_roi(image_path: str):
    img = cv2.imread(image_path)
    print(img.shape)
    Height, Width, Channel = img.shape

    cv2.namedWindow("Img_ROI")
    cv2.resizeWindow("Img_ROI", 640, 240)
    cv2.createTrackbar("x0", "Img_ROI", 0, Width, callback)  # 滑动条名称，所在窗口名称，初始值，最大值，回调函数
    cv2.createTrackbar("x1", "Img_ROI", 180, Width, callback)
    cv2.createTrackbar("y0", "Img_ROI", 0, Height, callback)
    cv2.createTrackbar("y1", "Img_ROI", 255, Height, callback)
    while True:
        x0 = cv2.getTrackbarPos("x0", "Img_ROI")
        x1 = cv2.getTrackbarPos("x1", "Img_ROI")
        y0 = cv2.getTrackbarPos("y0", "Img_ROI")
        y1 = cv2.getTrackbarPos("y1", "Img_ROI")
        mask = img[y0:y1, x0:x1]
        cv2.imshow("Img_mask", mask)

        if cv2.waitKey(1) == ord('q'):
            break
    cv2.destroyAllWindows()

In [4]:
get_roi('picture_0.jpg')

(480, 640, 3)


## 提取颜色范围

因为在RGB中间中，我们并不好将黄色和红色分离开来，所以我们后续的提取步骤需要将颜色通道转换到HSV空间进行。和上面ROI步骤一样，我们都需要通过滑动条进行参数设置。

In [5]:
import cv2
import numpy as np

def callback(a):
    pass

def get_object_hsv(image_path:str):
    cv2.namedWindow("HSV")
    cv2.resizeWindow("HSV", 640, 240)
    cv2.createTrackbar("H Min", "HSV", 0, 180, callback) #滑动条名称，所在窗口名称，初始值，最大值，回调函数
    cv2.createTrackbar("H Max", "HSV", 180, 180, callback)
    cv2.createTrackbar("S Min", "HSV", 0, 255, callback)
    cv2.createTrackbar("S Max", "HSV", 255, 255, callback)
    cv2.createTrackbar("V Min", "HSV", 0, 255, callback)
    cv2.createTrackbar("V Max", "HSV", 255, 255, callback)
    
    img = cv2.imread(image_path)
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    while True:
        h_min = cv2.getTrackbarPos("H Min", "HSV")  # 滑动条名称，窗口名称
        h_max = cv2.getTrackbarPos("H Max", "HSV")
        s_min = cv2.getTrackbarPos("S Min", "HSV")
        s_max = cv2.getTrackbarPos("S Max", "HSV")
        v_min = cv2.getTrackbarPos("V Min", "HSV")
        v_max = cv2.getTrackbarPos("V Max", "HSV")

        lower = np.array([h_min, s_min, v_min])
        upper = np.array([h_max, s_max, v_max])
        mask = cv2.inRange(hsv_img, lower, upper)
        cv2.imshow("Img", img)
        cv2.imshow('Mask', mask)

        if cv2.waitKey(1) == ord('q'):
            break
    cv2.destroyAllWindows()

In [6]:
get_object_hsv('picture_0.jpg')

## 传送带视频处理

通过上面的步骤，我们已经获取到了需要的ROI和颜色范围参数，我们接下来只需要将其应用到传送带视频上即可。

In [7]:
import os
import time
import cv2


def process_video(video_path: str, output_path: str, hsv_range: dict, roi_region: dict, threhold: float, FPS: int = 30):
    video_name = os.path.basename(video_path)

    yellow_num = 0
    yellow_flag = False
    red_num = 0
    red_flag = False

    cap = cv2.VideoCapture(video_path)
    video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    roi_WIDTH = roi_region["x1"] - roi_region["x0"]
    roi_HEIGHT = roi_region["y1"] - roi_region["y0"]
    roi_size = roi_WIDTH * roi_HEIGHT
    scale = 0.6

    red_object_img = cv2.imread('data/RedObject.jpg')
    red_img_height, red_img_width = red_object_img.shape[:2]
    red_img_height = int(red_img_height*scale)
    red_img_width = int(red_img_width*scale)
    red_object_img = cv2.resize(red_object_img, (red_img_width, red_img_height))


    yellow_object_img = cv2.imread('data/YellowObject.jpg')
    yellow_img_height, yellow_img_width = yellow_object_img.shape[:2]
    yellow_img_height = int(yellow_img_height*scale)
    yellow_img_width = int(yellow_img_width*scale)
    yellow_object_img = cv2.resize(yellow_object_img, (yellow_img_width,yellow_img_height))

    video_out = cv2.VideoWriter(output_path, cv2.VideoWriter.fourcc(*'mp4v'), FPS, (video_width, video_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Stream End!")
            break
        hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        roi_image = hsv_image[roi_region['y0']:roi_region['y1'], roi_region['x0']:roi_region['x1']]
        red_mask = ~cv2.inRange(roi_image, hsv_range['red'][0], hsv_range['red'][1])
        yellow_mask = cv2.inRange(roi_image, hsv_range['yellow'][0], hsv_range['yellow'][1])
        yellow_rate = (yellow_mask == 255).sum() / roi_size
        red_rate = (red_mask == 255).sum() / roi_size

        if yellow_flag and not (yellow_rate > threhold):
            yellow_num += 1
            yellow_flag = False
        if not yellow_flag:
            yellow_flag = (yellow_rate > threhold)

        if red_flag and not (red_rate > threhold):
            red_num += 1
            red_flag = False
        if not red_flag:
            red_flag = (red_rate > threhold)

        cv2.rectangle(frame, (roi_region["x0"], roi_region["y0"]), (roi_region["x1"], roi_region["y1"]), (0, 255, 0))
        cv2.putText(frame, "Detection_Region", (roi_region["x0"], roi_region["y0"]), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (255, 0, 0), 1)
        frame[10:10+red_img_height, 50:50 + red_img_width] = red_object_img
        frame[10:10+yellow_img_height, 200:200 + yellow_img_width] = yellow_object_img

        cv2.putText(frame, f"Red:{red_num}", (90, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(frame,f"Yellow:{yellow_num}",(240,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.imshow(f"{video_name}", frame) #平台无法运行该行代码

        video_out.write(frame)

        red_highlight = np.zeros_like(roi_image)
        red_highlight[red_mask == 255] = (0, 0, 255)
        yellow_highlight = np.zeros_like(roi_image)
        yellow_highlight[yellow_mask == 255] = (0, 255, 255)
        highlight_frame = red_highlight + yellow_highlight
        cv2.imshow(f"Highlight:{video_name}", highlight_frame) #平台无法运行该行代码

        time.sleep(1 / FPS)
        if cv2.waitKey(1) == ord('q'):
            break
    cap.release()
    video_out.release()
    cv2.destroyAllWindows()

def run():
    hsv_range = {
        'red': np.array([[10, 0, 0], [86, 255, 255]]),
        'yellow': np.array([[23, 18, 221], [45, 180, 255]])
    }
    roi_region = {
        'x0': 230,
        'x1': 350,
        'y0': 123,
        'y1': 319
    }
    threhold = 0.01
    video_list = ["data/Belt1.mp4", "data/Belt2.mp4"]
    
    for video_path in video_list:
        video_name = os.path.basename(video_path)
        output_video_path = "output" + "_" + video_name
        process_video(video_path, output_video_path, hsv_range, roi_region, threhold, 25)

In [8]:
run()

Stream End!
Stream End!


## 实验结果

在运行了上面这些实验后，我们能够在根目录下得到`output_Belt1.mp4`和`output_Belt2.mp4`这两个视频。我们通过格式工厂，将视频通过AVC(H264)编码格式重新编码，处理为`result_Belt1.mp4`和`result_Belt2.mp4`。

**Belt1.mp4处理后视频(data/result_Belt1.mp4)**

<video controls src="data/result_Belt1.mp4" />

**Belt2.mp4处理后视频(data/result_Belt2.mp4)**

<video controls src="data/result_Belt2.mp4" />